## YΣ19 Artificial Intelligence II
# Homework 4

### Iglezou Myrto - 111520170038


<img src="https://venturebeat.com/wp-content/uploads/2020/03/CORD-19.png?w=1200&strip=all" alt="Cord-19" width="600"/>

# Project Description



The objective of this project is about developing a document retrieval system to return titles of scientific papers containing the answer to a given user question. The dataset used in this exercise is from [COVID-19 Open Research Dataset (CORD-19)](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html), the first version. We are gonna implement 2 different sentence embedding approaches, in order for the model to retrieve the titles of the papers related to a given question.


# **Question 1** 

## Step 1 - Preprocess the provided dataset

### Read all json files from folder and keep for dataset the title and the body. Then save the dataframe to a csv file for faster reading of the dataframe.

In [ ]:
import io
import os
from google.colab import drive
import pandas as pd 
import json

drive.mount('/content/drive',force_remount=True)
path = r"/content/drive/My Drive/cord-19_2020-03-13/cord-19_2020-03-13/2020-03-13/comm_use_subset"

dataset_df = pd.DataFrame(columns=['id', 'title', 'body'])

for filename in os.listdir(path):
   with open(os.path.join(path, filename), 'r') as f:  
      json_text = json.load(f)

      id = json_text['paper_id']
      # print(id)
      title = json_text['metadata']['title']
      # print(title)
      body = json_text['body_text']
      # print(body)

      dataset_df.loc[len(dataset_df)] = [id,title,body]
   

Mounted at /content/drive


In [ ]:
dataset_df.to_csv('dataset.csv',index=False)
!cp dataset.csv "drive/My Drive/"

### Read the dataset from the csv file and save the information in a dataframe

In [53]:
import io
from google.colab import drive
import pandas as pd 
import sys 

drive.mount('/content/drive',force_remount=True)
filePath = r"/content/drive/My Drive/dataset.csv"
dataset_df = pd.read_csv(filePath)
dataset_df.title = dataset_df.title.astype(str)  # make everything str, for lower() function

Mounted at /content/drive


**Dataframe before the preprocess**

In [54]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,MINI REVIEW Acute Phase Proteins in Marine Mam...,[{'text': 'The mammalian immune system include...
1,14374db205f6934d9cba148624462000bc6ec7be,Antibody Treatment against Angiopoietin-Like 4...,[{'text': 'IMPORTANCE Despite extensive global...
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,Quantifying the seasonal drivers of transmissi...,"[{'text': ""Growing human population, urbanizat..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,Evaluation of the influenza-like illness surve...,[{'text': 'the first evaluation of the Tunisia...
4,1664a9df618ca74e099245a2bd65f3172aeac284,nan,"[{'text': 'Worldwide, lung cancer remains the ..."


Remove some of the special characters, such as [, { , ' , : and some words form json like 'text'.

In [55]:
def removeCharacters(x):
  x = x.str.replace(r'\'text\'', '')
  x = x.str.replace(r'\'start\'', '')
  x = x.str.replace(r'\'end\'', '')
  x = x.str.replace(r'[{}]', '')
  x = x.str.replace(r'[\[\]]', '')
  x = x.str.replace(r'["]', '')
  x = x.str.replace(r'[\']', '')
  x = x.str.replace(r'[:]', '')
  x = x.str.replace(r'[()]', '')
  return x

In [56]:
dataset_df['body'] = removeCharacters(dataset_df['body'])

Remove all the uppercase letters from title and body

In [57]:
dataset_df['body'] = dataset_df['body'].apply(lambda x: x.lower())
# dataset_df['title'] = dataset_df['title'].apply(lambda x: x.lower())

**Dataframe after the preprocess**

In [58]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,MINI REVIEW Acute Phase Proteins in Marine Mam...,the mammalian immune system includes innate o...
1,14374db205f6934d9cba148624462000bc6ec7be,Antibody Treatment against Angiopoietin-Like 4...,"importance despite extensive global efforts, ..."
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,Quantifying the seasonal drivers of transmissi...,"growing human population, urbanization and gl..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,Evaluation of the influenza-like illness surve...,the first evaluation of the tunisian influenz...
4,1664a9df618ca74e099245a2bd65f3172aeac284,nan,"worldwide, lung cancer remains the most frequ..."


In [59]:
%%capture
!pip install transformers

In [60]:
%%capture
!pip install sentence-transformers

In [61]:
import torch
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
    device = 'cuda'
else:
    print('No GPU available, training on CPU.')
    device = 'cpu'

No GPU available, training on CPU.


## Questions

In [62]:
questions =  [
                  "What are the coronoviruses?",
                  "What was discovered in Wuhuan in December 2019?",
                  "What is Coronovirus Disease 2019?",
                  "What is COVID-19?",
                  "What is caused by SARS-COV2?",
                  "How is COVID-19 spread?",
                  "Where was COVID-19 discovered?",
                  "How does coronavirus spread?",
              ]

## Create the list of sentences



In [63]:
import nltk
from nltk import tokenize
nltk.download('punkt')

ListOfBodies = dataset_df['body'].apply(lambda x: tokenize.sent_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
ListOfSentences = []
numOfArticles = 0
for text in ListOfBodies:
  ListOfSentences += text
  numOfArticles+=1
  if(numOfArticles == 10):
    break

## Usefull functions for the models

In [65]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(sentences, embeddings, query_embedding, k = 1):
    X = np.stack(embeddings)
    score_map = zip(sentences, cosine_similarity(X, query_embedding.reshape(1, -1)))
    return sorted(score_map, key=lambda v: v[1], reverse=True)[:k]

## Step 2.a - First sentence embedding approach

### Model

Uses Sentence-BERT (SBERT), a modification of the BERT network using siamese and triplet networks that is able to derive semantically meaningful sentence embeddings. This allows more efficient semantic search, which is utilized in the following application.

The siamese network architecture enables that fixed-sized vectors for input sentences can be derived. Using a similarity measure like cosine similarity or Manhatten / Euclidean distance, semantically similar sentences can be found. Cosine similarity is used in this work.

SBERT fine tuned on NLI data which creates SOTA sentence embeddings, as reported in the [SBERT paper](https://arxiv.org/pdf/1908.10084.pdf).

SBERT Framework example

<img src="https://combine.se/wp-content/uploads/2019/09/3.png" alt="Cord-19" width="300"/>

In [66]:
import torch

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens',device=device)

In [67]:
def ask_question(question,ListOfSentences,sentence_embeddings,dataset_df):
  query_vec = sbert_model.encode([question])[0]
  similar = most_similar(ListOfSentences,sentence_embeddings,query_vec)
  row = dataset_df[dataset_df['body'].str.contains(similar[0][0])]
  title = row['title'].tolist()[0]
  text = similar[0][0]

  return title,text

In [68]:
from termcolor import colored

def print_answer(question,title,text):

  print(colored("Question : ",attrs=['bold']),question)
  print("\n")
  print(colored("Title : ",attrs=['bold']),title)
  print("\n")
  print(colored("Text : ",attrs=['bold']),text)
  print("\n")
  # text = row['body'].apply(lambda x: tokenize.sent_tokenize(x))
  # num = 0
  # for t in text:
  #   for sentence in t:
  #     if sentence == l1:
  #         print(t[num-1], end=" ")
  #         print("\n")
  #         print(colored(sentence,'grey','on_yellow'), end=" ")
  #         print("\n")
  #         print(t[num+1], end=" ")
  #         num+=1
  # print("\n")

In [69]:
sentence_embeddings = sbert_model.encode(ListOfSentences,device=device)

In [73]:
for question in questions: 
  title,text = ask_question(question,ListOfSentences,sentence_embeddings,dataset_df)
  print("-------------------------------------------------------------------------")
  print_answer(question, title, text)

-------------------------------------------------------------------------
Question :  What are the coronoviruses?


Title :  Antibody Treatment against Angiopoietin-Like 4 Reduces Pulmonary Edema and Injury in Secondary Pneumococcal Pneumonia


Text :  3a and b and immunoneutralization of cangptl4  fig.


-------------------------------------------------------------------------
Question :  What was discovered in Wuhuan in December 2019?


Title :  ADAM17-dependent signaling is required for oncogenic human papillomavirus entry platform assembly


Text :  what triggers the assembly of the secondary entry receptor complex, that contains cd151 and egfr as functional components mikulicˇic´and florin, 2019 .


-------------------------------------------------------------------------
Question :  What is Coronovirus Disease 2019?


Title :  ADAM17-dependent signaling is required for oncogenic human papillomavirus entry platform assembly


Text :  what triggers the assembly of the secondary ent

## Step 2.b - Second sentence embedding approach

In [71]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.2M      0  0:00:14  0:00:14 --:--:-- 11.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  2086k      0  0:16:58  0:16:58 --:--:-- 2360k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


In [87]:
import models
from models import InferSent
import torch 

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

MessageError: ignored

In [ ]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [ ]:
model.build_vocab(sentences, tokenize=True)

## Step 3 - Comparison of two models